In [5]:
from flask import Flask, request, render_template
from binance.client import Client
from binance.enums import *
import pandas as pd
import json
import config
import binance_script as my_bnc

client_1 = Client(config.API_KEY_1, config.API_SECRET_1)
client_2 = Client(config.API_KEY_2, config.API_SECRET_2)

In [11]:
# receive webhook

wbhook = {
"passphrase":"Pooh_12345",
"symbol":"BTCUSDT",
"base_currency":"USDT",
"side":"SELL",
"buy_fixed_or_ratio":"fixed",
"sell_fixed_or_ratio":"ratio",
"buy_ratio_percent":"100",
"buy_fixed_amount_USDT":"20",
"sell_ratio_percent":"100",
"sell_fixed_amount_USDT":"20",
"take_profit_percent":"30",
"stop_loss_percent":"20",
"trailing_stop_(no/historical/callback)":"no",
"trailing_stop_activation_percent":"20",
"trailing_stop_historical_bar":"10",
"trailing_stop_callback_rate":"20"
}

In [12]:
##########################################################################
# recieve webhook
# wbhook = json.loads(request.data)

##########################################################################
# get webhook data

passphrase = wbhook['passphrase']
base_currency = wbhook['base_currency']

symbol = wbhook['symbol']
target_symbol = symbol.split(base_currency)[0]

side = wbhook['side']

buy_ratio_percent = float(wbhook['buy_ratio_percent'])
buy_fixed_amount_USDT = float(wbhook['buy_fixed_amount_USDT'])
buy_fixed_or_ratio = wbhook['buy_fixed_or_ratio']

sell_ratio_percent = float(wbhook['sell_ratio_percent'])
sell_fixed_amount_USDT = float(wbhook['sell_fixed_amount_USDT'])
sell_fixed_or_ratio = wbhook['sell_fixed_or_ratio']

take_profit_percent = float(wbhook['take_profit_percent'])
stop_loss_percent = float(wbhook['stop_loss_percent'])

trailing_stop_Type = wbhook['trailing_stop_(no/historical/callback)']
trailing_stop_activation_percent = float(wbhook['trailing_stop_activation_percent'])
trailing_stop_historical_bar = float(wbhook['trailing_stop_historical_bar'])
trailing_stop_callback_rate = float(wbhook['trailing_stop_callback_rate'])

##########################################################################
# check user
if passphrase == config.WEBHOOK_PASSPHRASE_1:
    client = client_1
    user_no = 1
elif passphrase == config.WEBHOOK_PASSPHRASE_2:
    client = client_2
    user_no = 2
else:
    client = "N/A"
    user_no = "N/A"

##########################################################################
# check account balance
base_balance = float(my_bnc.check_balance(base_currency, user_no))
target_balance = float(my_bnc.check_balance(target_symbol, user_no))

##########################################################################
# get all tick
all_price = client.get_all_tickers()
df_all_price = pd.DataFrame(all_price)

# check target price
price = df_all_price[(df_all_price.symbol == symbol)]
price = float(price['price'])

##########################################################################
# Position Sizing
## BUY
if buy_fixed_or_ratio == "fixed":
    buy_amount = buy_fixed_amount_USDT / price

elif buy_fixed_or_ratio == "ratio":
    buy_amount = (base_balance * buy_ratio_percent) / price

else:
    buy_amount = 0

## SELL
if sell_fixed_or_ratio == "fixed":
    sell_amount = sell_fixed_amount_USDT / price

elif sell_fixed_or_ratio == "ratio":
    sell_amount = (target_balance * sell_ratio_percent) / 100

else:
    sell_amount = 0
    
##########################################################################
# select order amount

if side == "BUY":
    order_amount = round(buy_amount,5)
elif side == "SELL":
    order_amount = round(sell_amount,5)
else:
    order_amount = "N/A"

##########################################################################
# order execution

order_response = client.create_order(symbol=symbol, side=side, type=ORDER_TYPE_MARKET, quantity=order_amount)


USDT: 87.69618583
BTC: 0.00087


In [13]:
print(order_response)

{'symbol': 'BTCUSDT', 'orderId': 11786414599, 'orderListId': -1, 'clientOrderId': 'BWHUTsQaRvasZ0uEG23Lra', 'transactTime': 1658465184265, 'price': '0.00000000', 'origQty': '0.00087000', 'executedQty': '0.00087000', 'cummulativeQuoteQty': '19.98235140', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '22968.22000000', 'qty': '0.00087000', 'commission': '0.00000000', 'commissionAsset': 'BNB', 'tradeId': 1505169498}]}


In [3]:
# get webhook

passphrase = wbhook['passphrase']
base_currency = wbhook['base_currency']

symbol = wbhook['symbol']
target_symbol = symbol.split(base_currency)[0]

side = wbhook['side']

buy_ratio_percent = float(wbhook['buy_ratio_percent'])
buy_fixed_amount_USDT = float(wbhook['buy_fixed_amount_USDT'])
buy_fixed_or_ratio = wbhook['buy_fixed_or_ratio']

sell_ratio_percent = float(wbhook['sell_ratio_percent'])
sell_fixed_amount_USDT = float(wbhook['sell_fixed_amount_USDT'])
sell_fixed_or_ratio = wbhook['sell_fixed_or_ratio']

take_profit_percent = float(wbhook['take_profit_percent'])
stop_loss_percent = float(wbhook['stop_loss_percent'])

trailing_stop_Type = wbhook['trailing_stop_(no/historical/callback)']
trailing_stop_activation_percent = float(wbhook['trailing_stop_activation_percent'])
trailing_stop_historical_bar = float(wbhook['trailing_stop_historical_bar'])
trailing_stop_callback_rate = float(wbhook['trailing_stop_callback_rate'])

#################################################################################################
# check user
if passphrase == config.WEBHOOK_PASSPHRASE_1:
    client = Client(config.API_KEY_1, config.API_SECRET_1)
    user_no = 1
elif passphrase == config.WEBHOOK_PASSPHRASE_2:
    client = Client(config.API_KEY_2, config.API_SECRET_2)
    user_no = 2
else:
    user_no = "N/A"

#################################################################################################
# check account balance
base_balance = float(my_bnc.check_balance(base_currency, user_no))
target_balance = float(my_bnc.check_balance(target_symbol, user_no))

#################################################################################################
# get all tick
all_price = client.get_all_tickers()
df_all_price = pd.DataFrame(all_price)

# check target price
price = df_all_price[(df_all_price.symbol == symbol)]
price = float(price['price'])

#################################################################################################
# Position Sizing
## BUY
if buy_fixed_or_ratio == "fixed":
    buy_amount = buy_fixed_amount_USDT / price

elif buy_fixed_or_ratio == "ratio":
    buy_amount = (base_balance * buy_ratio_percent) / price

else:
    buy_amount = 0

## SELL
if sell_fixed_or_ratio == "fixed":
    sell_amount = sell_fixed_amount_USDT / price

elif sell_fixed_or_ratio == "ratio":
    sell_amount = (target_balance * sell_ratio_percent) / 100

else:
    sell_amount = 0
    
#################################################################################################
# select order amount

if side == "BUY":
    order_amount = round(buy_amount,5)
elif side == "SELL":
    order_amount = round(sell_amount,5)
else:
    order_amount = "N/A"

#################################################################################################
# order execution

order = client.create_order(symbol=symbol, side=side, type=ORDER_TYPE_MARKET, quantity=order_amount)
print(order)

KeyError: 'base_currency'

In [ ]:
order_response = client.create_order(symbol=symbol, side=side, type=ORDER_TYPE_MARKET, quantity=order_amount)
print(order_response)

if order_response:
    return {
        "code" : "Success",
        "message" : "Order Executed",
        "symbol" : symbol,
        "side" : side,
        "amount" : order_amount
    }
else:
    print("Order Failed")
    return{
        "code" : "Error",
        "message" : "Order Failed",
        "symbol" : symbol,
        "side" : side,
        "amount" : order_amount
    }

In [6]:
print(order_response)

NameError: name 'order_response' is not defined